In [31]:
import time
import json
import numpy as np
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt
import json
import ast
import scipy.stats as stats
import stanza
import math
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
# import SVC classifier
from sklearn.svm import SVC
# import metrics to compute accuracy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [58]:
final_df = pd.read_csv('./finalwithDate.csv')

In [61]:
# final_df = final_df.drop(['Unnamed: 0','reason', 'trafficSource.keyword', 'hits'], axis = 1)
final_df

,Unnamed: 0,date,device.deviceCategory,geoNetwork.country,totals.hits,totals.pageviews,totals.timeOnSite,trafficSource.keyword,hits,NLPValue,isAnomaly,reason,norm_country
0,0,20180501,desktop,United States,10,9.0,416.0,NaN,"[{'hitNumber': '14', 'time': '0', 'hour': '0',...",1.0,0.0,NaN,0
1,1,20180501,mobile,United States,1,1.0,0.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '0', ...",1.0,0.0,NaN,0
2,2,20180501,desktop,India,1,1.0,0.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '0', ...",1.0,0.0,NaN,1
3,3,20180501,desktop,United States,53,34.0,682.0,NaN,"[{'hitNumber': '1', 'time': '0', 'hour': '0', ...",1.0,0.0,NaN,0
4,4,20180501,mobile,Nigeria,1,1.0,0.0,(not set),"[{'hitNumber': '2', 'time': '0', 'hour': '0', ...",1.0,0.0,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401584,401584,20181015,desktop,South Korea,5,4.0,45.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",1.0,0.0,NaN,17
401585,401585,20181015,desktop,India,7,7.0,266.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",1.0,0.0,NaN,1
401586,401586,20181015,desktop,United States,5,5.0,39.0,NaN,"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",1.0,0.0,NaN,0
401587,401587,20181015,desktop,Taiwan,6,4.0,157.0,(not provided),"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",1.0,0.0,NaN,8


In [60]:
final_df['norm_country'] = pd.factorize(final_df['geoNetwork.country'])[0]

In [62]:
final_df['isAnomaly'].fillna(value = 0, inplace = True)
aomalousSessions = final_df[final_df['isAnomaly'] == 1]
len(aomalousSessions)

966

In [ ]:
final_df['isAnomaly'].fillna(value = 0, inplace = True)
nonAomalousSessions = final_df[final_df['isAnomaly'] == 0]
nonAomalousSessions

In [ ]:
sampled_anomaly_train = aomalousSessions.sample(frac =.50)
sampled_anomaly_test = pd.concat([aomalousSessions, sampled_anomaly_train, sampled_anomaly_train]).drop_duplicates(keep=False)
sampled_anomaly_train

In [ ]:
sampled_nonAnomaly_train = nonAomalousSessions.sample(frac =.10)
sampled_nonAnomaly_test = pd.concat([nonAomalousSessions, sampled_nonAnomaly_train]).drop_duplicates(keep=False)
sampled_nonAnomaly_test

In [66]:
train_data = pd.concat([sampled_anomaly_train, sampled_nonAnomaly_train])
train_data

,Unnamed: 0,date,device.deviceCategory,geoNetwork.country,totals.hits,totals.pageviews,totals.timeOnSite,trafficSource.keyword,hits,NLPValue,isAnomaly,reason,norm_country
56680,56680,20180521,desktop,India,1,1.0,0.0,googlemerchandisestore.html,"[{'hitNumber': '1', 'time': '0', 'hour': '2', ...",1.0,1.0,"Search Keywords having SQL Injection, XSS Scri...",1
25492,25492,20180509,mobile,(not set),15,10.0,602.0,(not provided),"[{'hitNumber': '1', 'time': '0', 'hour': '18',...",1.0,1.0,total hits of a session from NON Opearting/rar...,60
23025,23025,20180509,mobile,Poland,500,251.0,2754.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '3', ...",1.0,1.0,total pageViews from a session is absurdly hig...,22
7095,7095,20180503,desktop,United States,54,36.0,2579.0,NaN,"[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",1.0,1.0,user generating unusual higher volume of traff...,0
21496,21496,20180508,desktop,Ukraine,35,24.0,404.0,(not provided),"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",1.0,1.0,user generating unusual higher volume of traff...,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57874,57874,20180521,desktop,United States,2,2.0,20.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",1.0,0.0,NaN,0
138546,138546,20180621,desktop,United States,1,1.0,0.0,(not provided),"[{'hitNumber': '1', 'time': '0', 'hour': '0', ...",1.0,0.0,NaN,0
6576,6576,20180503,mobile,Hong Kong,3,3.0,216.0,(not provided),"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",1.0,0.0,NaN,54
186788,186788,20180712,desktop,Colombia,1,1.0,0.0,(not set),"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",1.0,0.0,NaN,66


In [67]:
test_data = pd.concat([sampled_nonAnomaly_test, sampled_anomaly_test])

In [69]:
test_data.to_csv('./testData.csv')

In [ ]:
#SMOTE Application
pd.value_counts(train_data['isAnomaly']).plot.bar()
plt.title('Anomaly class histogram')
plt.xlabel('isAnomaly')
plt.ylabel('Frequency')
train_data['isAnomaly'].value_counts()



In [70]:
X_train = train_data.drop(['isAnomaly', 'Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
y_train = train_data['isAnomaly']

In [71]:
X_test = test_data.drop(['isAnomaly', 'Unnamed: 0', 'device.deviceCategory', 'date', 'geoNetwork.country', 'trafficSource.keyword', 'hits', 'reason'], axis=1)
y_test = test_data['isAnomaly']
y_test

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
392778    1.0
396401    1.0
397918    1.0
399602    1.0
401557    1.0
Name: isAnomaly, Length: 361044, dtype: float64

In [ ]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [72]:
# instantiate classifier with default hyperparameters
svc=SVC() 


# fit classifier to training set
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with default hyperparameters: 0.9987


In [73]:
import pickle
from joblib import Parallel, delayed
import joblib
joblib.dump(svc, 'svm.pkl')

['svm.pkl']

In [74]:
print('Training set score: {:.4f}'.format(svc.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(svc.score(X_test, y_test)))

Training set score: 0.9883
Test set score: 0.9987


In [ ]:
# clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(svc, X_train, y_train, cv=5)
# print("%0.2f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))
scores

In [39]:
#Logistic Regression Starts
logreg = LogisticRegression(solver='liblinear', random_state=0)
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [48]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 0.9864


In [50]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier



# instantiate the classifier 

rfc = RandomForestClassifier(n_estimators=100, random_state=0)



# fit the model

rfc.fit(X_train, y_train)



# Predict the Test set results

y_pred = rfc.predict(X_test)

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.9914
